## Classe de fusão

Pega o vetor 1D, repete ele na grade 8x8 e concatena com o vídeo

In [1]:
import torch
import torch.nn as nn

class SpatialBroadcastFuser(nn.Module):
    def __init__(self, height=8, width=8):
        super().__init__()
        self.height = height
        self.width = width

    def forward(self, z_visual, z_action):
        """
        z_visual: Tensor (Batch, Canais_V, H, W) 
        z_action: Tensor (Batch, Features_A)     
        """
        
        # Expandir dimensões da ação para (Batch, Features, 1, 1)
        z_action_expanded = z_action.unsqueeze(2).unsqueeze(3)
        
        # Preenche a grade (H, W), mantém batch e features do mesmo tamanho
        z_action_tiled = z_action_expanded.expand(-1, -1, self.height, self.width)
        
        # Concatenar no eixo dos canais (dim=1)
        z_fused = torch.cat([z_visual, z_action_tiled], dim=1)
        
        return z_fused

# Simula dados
batch_size = 32
z_v = torch.randn(batch_size, 8, 8, 8)  
z_a = torch.randn(batch_size, 16)       

fuser = SpatialBroadcastFuser(height=8, width=8)
z_out = fuser(z_v, z_a)

print(f"Visual Shape: {z_v.shape}")
print(f"Action Shape: {z_a.shape}")
print(f"Fused Shape:  {z_out.shape}") # [32, 24, 8, 8]

Visual Shape: torch.Size([32, 8, 8, 8])
Action Shape: torch.Size([32, 16])
Fused Shape:  torch.Size([32, 24, 8, 8])


## Teste de qualidade 

Passando o latente final, ele consegue separar a ação original?

In [2]:
import torch.optim as optim

# Decodificador simples de teste
# Pega o tensor fundido, faz uma média global e tenta adivinhar a ação
class Probe(nn.Module):
    def __init__(self, in_channels=24, action_dim=16):
        super().__init__()
        # Pega a média de cada canal (reduz 8x8 para 1x1)
        self.gap = nn.AdaptiveAvgPool2d(1) 
        # Tenta recuperar o vetor original
        self.linear = nn.Linear(in_channels, action_dim)

    def forward(self, x):
        x = self.gap(x)        # [B, 24, 8, 8] -> [B, 24, 1, 1]
        x = x.flatten(1)       # [B, 24]
        return self.linear(x)  # [B, 16]

# Teste
probe = Probe(in_channels=24, action_dim=16)
optimizer = optim.Adam(probe.parameters(), lr=0.01)
criterion = nn.MSELoss()


# Loop de treino rápido
for i in range(500):
    optimizer.zero_grad()
    
    # Gera fusão
    z_fused = fuser(z_v, z_a).detach() 
    
    # Tenta recuperar a ação
    z_a_recovered = probe(z_fused)
    
    # Compara com a ação original
    loss = criterion(z_a_recovered, z_a)
    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print(f"Iter {i}: Loss = {loss.item():.6f}")

if loss.item() < 0.01:
    print("\nA fusão preservou a informação")
else:
    print("O modelo não conseguiu encontrar a ação dentro do tensor fundido")

Iter 0: Loss = 1.051430
Iter 100: Loss = 0.041269
Iter 200: Loss = 0.006832
Iter 300: Loss = 0.002940
Iter 400: Loss = 0.001650

A fusão preservou a informação
